In [1]:
!pip install wget
import os
import torch
import pandas as pd
import numpy as np
import torch
from torch import nn


ssh_config = """
Host github.com
  IdentityFile ~/.ssh/github.pem
  User davipeag
  StrictHostKeyChecking no
"""

if os.name == 'nt':
  base_path = ""
  REPO_DIR = "."
  STORE_DIR ="." 
  print("Windows")
else:
  print("Unix-like")
  REPO_DIR = "/tmp/HeartRateRegression"
  from google.colab import drive
  drive.mount('/content/drive')
  GIT_PATH = "/content/drive/My\ Drive/deeplearning_project/github.pem"
  DATA_DIR = os.path.join(REPO_DIR, "repo")
  STORE_DIR ="/content/drive/My Drive/deeplearning_project/" 
  !mkdir ~/.ssh
  !cp -u {GIT_PATH} ~/.ssh/
  !chmod u=rw,g=,o= ~/.ssh/github.pem
  !echo "{ssh_config}" > ~/.ssh/config
  !chmod u=rw,g=,o= ~/.ssh/config
  ! (cd /tmp && git clone git@github.com:davipeag/HeartRateRegression.git)
  ! (cd {REPO_DIR} && git pull )
  import sys
  sys.path.append(REPO_DIR)

def git_pull():
  ! (cd {REPO_DIR} && git pull )

git_pull()

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=267649f1d059681a9db94d9285e16618d54e0eb24a64c23cfdcac5aae727aaa9
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Unix-like
Mounted at /content/drive
Cloning into 'HeartRateRegression'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (248/248), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 633 (delta 156), reused 142 (delta 61), pack-reused 385
Receiving objects: 100% (633/633), 87.92 MiB | 36.04 MiB/s, done.
Resolving deltas: 100% (383/383), done.
Already up to date.
Already up to date.


In [2]:
import random
import torch
from torch import nn
import numpy as np
args = {
    'epoch_num': 250,     # Number of epochs.
    'lr': 1.0e-3,           # Learning rate.
    'weight_decay': 10e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 0,     # Number of workers on data loader.
    'batch_size': 128,     # Mini-batch size. 128
    'batch_test': 248,     # size of test batch
    'window': 15,
    'initial_window':5,
    'clip_norm': 6.0,     # Upper limit on gradient L2 norm ###
}
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

SEED = 1234
def reset_seeds():
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

reset_seeds()

cuda


In [3]:
from data_utils import (FormatPPGDalia, WesadExtractor)

SUBJECTS = [1,2,3,4,5,6,7,8,9,10,11,13,14,15,16,17]

extractor = WesadExtractor(DATA_DIR)
ppg_dalia_formatter = FormatPPGDalia()
dfs_train = [ppg_dalia_formatter.transform(extractor.extract_subject(i)) for i in SUBJECTS]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


In [133]:
git_pull()

import importlib

import PPG

from PPG import FullTrainer

importlib.reload(PPG.AttentionDefaults)
importlib.reload(PPG)
importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(PPG.Models)
importlib.reload(PPG.NoHrPceLstmModel)
importlib.reload(PPG.TrainerXY)
importlib.reload(PPG.TrainerIS)
importlib.reload(PPG.FullTrainer)


remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From github.com:davipeag/HeartRateRegression
   e268b7e..6254e97  master     -> origin/master
Updating e268b7e..6254e97
Fast-forward
 PPG/FullTrainer.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


<module 'PPG.FullTrainer' from '/tmp/HeartRateRegression/PPG/FullTrainer.py'>

In [ ]:
import matplotlib.pyplot as plt
# fchoice = {'val_sub': 4,
#   'ts_sub': 0,
#   'batch_size': 64,
#   'weight_decay': 0,
#   'lr': 0.001,
#   'nattrs': 5,
#   'bvp_count': 16,
#   'dropout_rate': 0.25,
#   'lstm_input': 128,
#   'lstm_size': 64,
#   'ts_h_size': 64
#   }
def compute_ensemble(results):
  ps = [v["predictions"][1].reshape(-1).numpy() for v in results]
  ys = [v["predictions"][0].reshape(-1).numpy() for v in results]

  for i in range(1, len(ys)-1):
    assert np.all(ys[i] == ys[i-1])

  s = ps[0]
  for p in ps[1:]:
    s = s + p

  a = s/len(ps)
  y = ys[0]

  plt.plot(a)
  plt.plot(y)

  return np.mean(np.abs(a - y))


fchoice = {'val_sub': 4,
  'ts_sub': 0,
  'batch_size': 64,
  'weight_decay': 0.0001,
  'lr': 0.001,
  'nattrs': 5,
  'bvp_count': 12,
  'dropout_rate': 0,
  'lstm_input': 128,
  'lstm_size': 32,
  'ts_h_size': 32
  }



from PPG import UtilitiesDataXY
aresults = list()
for ts_sub in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]:
  dresults = list()
  for i in range(7):
    full_trainer = FullTrainer.JointValNoHrPceLstmFullTrainer(dfs_train, args["device"])
    try:
      fchoice["ts_sub"] = ts_sub
      fchoice["val_sub"] = val_sub
      out = full_trainer.train(**fchoice)
      print(out["args"], out["metric"])
      dresults.append(out)
    except RuntimeError as e:
      if isinstance(e, KeyboardInterrupt):
        raise e
      else:
        print("####")
        print(f"Failed: {choice}")
        print("###")
  print(f"TS:{compute_ensemble(dresults)}")
  aresults.append(dresults)



best val epoch: 1
[1/40]: loss_train: 16.654 loss_val 17.112 loss_ts 16.085
best val epoch: 2
[2/40]: loss_train: 15.837 loss_val 16.354 loss_ts 17.822
best val epoch: 3
[3/40]: loss_train: 13.933 loss_val 14.349 loss_ts 16.677
best val epoch: 4
[4/40]: loss_train: 13.136 loss_val 13.472 loss_ts 12.371
best val epoch: 5
[5/40]: loss_train: 12.213 loss_val 12.730 loss_ts 16.494
best val epoch: 6
[6/40]: loss_train: 11.003 loss_val 11.755 loss_ts 12.812
best val epoch: 8
[8/40]: loss_train: 10.053 loss_val 10.333 loss_ts 8.684
best val epoch: 10
[10/40]: loss_train: 8.646 loss_val 9.223 loss_ts 8.673
best val epoch: 11
[11/40]: loss_train: 7.978 loss_val 8.663 loss_ts 8.160
best val epoch: 12
[12/40]: loss_train: 7.369 loss_val 8.288 loss_ts 10.536
best val epoch: 13
[13/40]: loss_train: 7.063 loss_val 7.903 loss_ts 10.549
best val epoch: 14
[14/40]: loss_train: 6.541 loss_val 7.370 loss_ts 8.106
best val epoch: 15
[15/40]: loss_train: 6.085 loss_val 6.895 loss_ts 7.285
best val epoch: 1

In [109]:

def compute_ensemble(results):
  ps = [v["predictions"][1].reshape(-1).numpy() for v in results]
  ys = [v["predictions"][0].reshape(-1).numpy() for v in results]

  for i in range(1, len(ys)-1):
    assert np.all(ys[i] == ys[i-1])

  s = ps[0]
  for p in ps[1:]:
    s = s + p

  a = s/len(ps)
  y = ys[0]

  plt.plot(a)
  plt.plot(y)

  np.mean(np.abs(a - y))

compute_ensemble(dresults)

# import matplotlib.pyplot as plt


# p = p.reshape(-1)

# plt.plot(y)
# plt.plot(p)


TypeError: ignored

In [79]:
import random 
options = {
  "ts_h_size": [64, 128],
  "lstm_size": [64, 64, 128],
  "lstm_input": [64, 128, 256],
  "dropout_rate": [0.25],
  "bvp_count": [8,16],
  "nattrs": [5],
  'lr': [0.001],
  'weight_decay': [0, 0.0001],
  'batch_size': [64, 128, 256],
  'ts_sub': [0],
  'val_sub': [4]
 }

def choose(options):
  choice = dict()
  for k,v in options.items():
    choice[k] = random.choice(v)
  return choice


In [72]:
results = list()
from PPG import UtilitiesDataXY
while True:
  full_trainer = FullTrainer.NoHrPceLstmFullTrainer(dfs_train, args["device"])
  choice = choose(options)
  try:
    out = full_trainer.train(**choice)
    print(out["args"], out["metric"])
    results.append([out["args"], out["metric"]])
  except RuntimeError as e:
    if isinstance(e, KeyboardInterrupt):
      raise e
    else:
      print("####")
      print(f"Failed: {choice}")
      print("###")

best val epoch: 1
[1/30]: loss_train: 14.871 loss_val 40.098 loss_ts 15.313
best val epoch: 4
[4/30]: loss_train: 13.214 loss_val 38.335 loss_ts 15.366
best val epoch: 9
[9/30]: loss_train: 10.959 loss_val 36.594 loss_ts 13.207
best val epoch: 11
[11/30]: loss_train: 10.100 loss_val 34.958 loss_ts 11.145
best val epoch: 13
[13/30]: loss_train: 9.185 loss_val 28.861 loss_ts 11.032
best val epoch: 14
[14/30]: loss_train: 8.935 loss_val 27.741 loss_ts 8.967
best val epoch: 15
[15/30]: loss_train: 8.144 loss_val 22.849 loss_ts 9.442
best val epoch: 16
[16/30]: loss_train: 8.264 loss_val 20.660 loss_ts 7.605
best val epoch: 17
[17/30]: loss_train: 7.284 loss_val 17.109 loss_ts 7.682
best val epoch: 18
[18/30]: loss_train: 6.850 loss_val 14.479 loss_ts 8.892
best val epoch: 24
[24/30]: loss_train: 5.647 loss_val 13.622 loss_ts 6.544
best val epoch: 26
[26/30]: loss_train: 5.922 loss_val 12.636 loss_ts 6.277
best val epoch: 28
[28/30]: loss_train: 5.360 loss_val 12.493 loss_ts 6.382
Final: 6.

KeyboardInterrupt: ignored

In [24]:
results

[[{'batch_size': 64,
   'bvp_count': 8,
   'dropout_rate': 0.5,
   'lr': 0.001,
   'lstm_input': 64,
   'lstm_size': 64,
   'nattrs': 5,
   'ts_h_size': 16,
   'ts_sub': 0,
   'val_sub': 4,
   'weight_decay': 0},
  9.884767],
 [{'batch_size': 128,
   'bvp_count': 8,
   'dropout_rate': 0,
   'lr': 0.001,
   'lstm_input': 64,
   'lstm_size': 32,
   'nattrs': 5,
   'ts_h_size': 16,
   'ts_sub': 0,
   'val_sub': 4,
   'weight_decay': 0.0001},
  7.2431717],
 [{'batch_size': 64,
   'bvp_count': 4,
   'dropout_rate': 0.25,
   'lr': 0.001,
   'lstm_input': 64,
   'lstm_size': 16,
   'nattrs': 5,
   'ts_h_size': 64,
   'ts_sub': 0,
   'val_sub': 4,
   'weight_decay': 0.0001},
  6.5298805],
 [{'batch_size': 128,
   'bvp_count': 8,
   'dropout_rate': 0.25,
   'lr': 0.001,
   'lstm_input': 256,
   'lstm_size': 64,
   'nattrs': 5,
   'ts_h_size': 128,
   'ts_sub': 0,
   'val_sub': 4,
   'weight_decay': 0},
  5.839752],
 [{'batch_size': 128,
   'bvp_count': 8,
   'dropout_rate': 0.25,
   'lr': 0.001

In [80]:
fchoice = {'val_sub': 4,
  'ts_sub': 0,
  'batch_size': 64,
  'weight_decay': 0,
  'lr': 0.001,
  'nattrs': 5,
  'bvp_count': 16,
  'dropout_rate': 0.25,
  'lstm_input': 128,
  'lstm_size': 64,
  'ts_h_size': 64}


dresults = list()
from PPG import UtilitiesDataXY
ts_sub = 3
for val_sub in [i for i in range(15) if i != ts_sub]:
  full_trainer = FullTrainer.NoHrPceLstmFullTrainer(dfs_train, args["device"])
  try:
    fchoice["ts_sub"] = ts_sub
    fchoice["val_sub"] = val_sub
    out = full_trainer.train(**fchoice)
    print(out["args"], out["metric"])
    dresults.append([out])
  except RuntimeError as e:
    if isinstance(e, KeyboardInterrupt):
      raise e
    else:
      print("####")
      print(f"Failed: {choice}")
      print("###")

best val epoch: 3
[3/30]: loss_train: 15.471 loss_val 10.642 loss_ts 9.652
best val epoch: 5
[5/30]: loss_train: 12.919 loss_val 10.403 loss_ts 10.214
best val epoch: 7
[7/30]: loss_train: 11.084 loss_val 9.540 loss_ts 11.058
best val epoch: 8
[8/30]: loss_train: 11.767 loss_val 8.751 loss_ts 12.954
best val epoch: 9
[9/30]: loss_train: 10.947 loss_val 8.107 loss_ts 12.848
best val epoch: 10
[10/30]: loss_train: 10.812 loss_val 7.675 loss_ts 13.429
best val epoch: 11
[11/30]: loss_train: 10.724 loss_val 7.331 loss_ts 13.989
best val epoch: 12
[12/30]: loss_train: 8.434 loss_val 6.469 loss_ts 11.949
best val epoch: 14
[14/30]: loss_train: 7.489 loss_val 6.221 loss_ts 12.631
best val epoch: 18
[18/30]: loss_train: 6.551 loss_val 5.858 loss_ts 10.856
best val epoch: 19
[19/30]: loss_train: 6.579 loss_val 5.526 loss_ts 10.445
best val epoch: 25
[25/30]: loss_train: 5.452 loss_val 5.403 loss_ts 8.816
best val epoch: 28
[28/30]: loss_train: 5.216 loss_val 5.307 loss_ts 8.529
Final: 8.5291595

TypeError: ignored

In [ ]:
y = 

In [ ]:
full_trainer = FullTrainer.AttentionFullTrainer(dfs_train, args["device"], 0, 1)

full_trainer.train()

In [ ]:
from PPG import UtilitiesDataXY 


transformers = PPG.AttentionDefaults.get_preprocessing_transformer()
make_loaders = UtilitiesDataXY.DataLoaderFactory(transformers, dfs_train).make_loaders

loader_tr, loader_val, loader_ts = make_loaders(ts_sub=0, val_sub=1)

In [ ]:
from PPG.Models import SnippetConvolutionalTransformer

net = SnippetConvolutionalTransformer().to(args["device"])

# x,y = next(iter(loader_tr))

# p = net(x)

criterion = nn.MSELoss().to(args["device"])# nn.L1Loss().to(args["device"]) #nn.CrossEntropyLoss().to(args["device"])
optimizer = torch.optim.Adam(net.parameters(), lr=args["lr"],
                             weight_decay=args["weight_decay"])


In [ ]:
from PPG.TrainerXY import (EpochTrainerXY, MetricsComputerXY, TrainHelperXY)
from preprocessing_utils import ZTransformer2

epoch_trainer = EpochTrainerXY(net, optimizer, criterion, args["device"])
ztransformer = ZTransformer2(['heart_rate', 'wrist-ACC-0', 'wrist-ACC-1', 'wrist-ACC-2',
              'wrist-BVP-0', 'wrist-EDA-0', 'wrist-TEMP-0', 'chest-ACC-0',
              'chest-ACC-1', 'chest-ACC-2', 'chest-Resp-0'])
metrics_comuter = MetricsComputerXY(ztransformer)

train_helper = TrainHelperXY(epoch_trainer, loader_tr, loader_val, loader_ts, metrics_comuter.mae)

In [ ]:
train_helper.train(30)